# Imported Libraries


In [3]:
!pip install kaggle pandas psycopg2 sqlalchemy

  Using cached kaggle-1.6.17-py3-none-any.whl
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/29/d4/1244ab8edf173a10fd601f7e13b9566c1b525c4f365d6bee918e68381889/pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for psycopg2 from https://files.pythonhosted.org/packages/66/de/baed128ae0fc07460d9399d82e631ea31a1f171c0c4ae18f9808ac6759e3/psycopg2-2.9.10-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/b7/ed/f6cd9395e41bfe47dd253d74d2dfc3cab34980d4e20c8878cb1117306085/SQLAlchemy-2.0.36-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for certifi>=2023.7.22 from https://files.pythonhosted.org/packages/12/90/3c9ff0512038035f59d279fddeb79f5f1eccd8859f06d6163c58798b9487/certifi-2024.8.30-py3-none-any.whl.metadata
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/f9/9b/335f97642


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install kagglehub


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for kagglehub from https://files.pythonhosted.org/packages/3c/74/10ea2151b11f8591f82d39c70c1dcc43d0160ba41f8ddb644eb1f5eac348/kagglehub-0.3.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
   ---------------------------------------- 43.0/43.0 kB 2.0 MB/s eta 0:00:00


In [7]:
import kagglehub
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import requests

c:\Users\eblow\.virtualenvs\backend-_O7a6dm_\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
host = 'localhost'  # Use explicit IP address
port = '5432'
database = 'postgres'
user = 'postgres'
password = 'Ebola101!'

connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'

# Establish the connection
engine = create_engine(connection_string)

try:
    with engine.connect() as connection:
        print("Connected to PostgreSQL database successfully!")
except Exception as e:
    print("Error connecting to PostgreSQL database:", e)

Connected to PostgreSQL database successfully!


# Extract

In [10]:
path = kagglehub.dataset_download("andykrause/kingcountysales")

print("Path to dataset files:", path)

100%|██████████| 119M/119M [00:02<00:00, 52.3MB/s] 

Extracting files...


Path to dataset files: C:\Users\eblow\.cache\kagglehub\datasets\andykrause\kingcountysales\versions\7


In [11]:
path2 = kagglehub.dataset_download("andykrause/king-county-homes")

print("Path to dataset files:", path2)

100%|██████████| 63.1M/63.1M [00:02<00:00, 25.0MB/s]

Extracting files...


Path to dataset files: C:\Users\eblow\.cache\kagglehub\datasets\andykrause\king-county-homes\versions\2


In [12]:
kingco_sales_df = pd.read_csv(path + "/kingco_sales.csv")

In [14]:
kingco_index = pd.read_csv("ATNHPIUS53033A.csv")

In [15]:
kingco_homes_df = pd.read_csv(path2 + "/kingco_homes.csv")

# Transform

Transforming the kingco_sales_df 'sale_date' column to sale_year and kingco_index 'DATE' column to 'year'

In [16]:
kingco_sales_df['sale_year'] = pd.to_datetime(kingco_sales_df['sale_date']).dt.year

In [17]:
kingco_index['year'] = pd.to_datetime(kingco_index['DATE']).dt.year

Renaming the 'ATNHPIUS53033A' to 'HPI'

In [18]:
kingco_index.rename(columns={'ATNHPIUS53033A': 'HPI'}, inplace=True)

Create the HPI mapping from kingco_index

In [19]:
hpi_mapping = kingco_index.set_index('year')['HPI'].to_dict()

In [20]:
# Determine the latest year in kingco_index and its HPI
latest_year = kingco_index['year'].max()
latest_hpi = kingco_index[kingco_index['year'] == latest_year]['HPI'].iloc[0]

 Adjust sale_price using the HPI ratio (latest year HPI / sale_year HPI)

In [21]:
kingco_sales_df['adjusted_sale_price'] = kingco_sales_df.apply(
    lambda row: row['sale_price'] * (latest_hpi / hpi_mapping[row['sale_year']]), axis=1
)

In [22]:
kingco_sales_df.drop(columns=['sale_price'], inplace=True)

generalizing the zonings to their types

In [23]:
zoning_to_type = {
    'SF 5000': 'Residential',
    'RA2.5P': 'Residential',
    'R6': 'Residential',
    'R15 OP': 'Residential',
    'SF 7200': 'Residential',
    'R15': 'Residential',
    'RA10': 'Residential',
    'RS 8': 'Residential',
    'R4P': 'Residential',
    'SF-S': 'Residential',
    'R-5': 'Residential',
    'R 4': 'Residential',
    'R-6': 'Residential',
    'R-8.4': 'Residential',
    'RS 10000': 'Residential',
    'R-15': 'Residential',
    'R1P': 'Residential',
    'RA5P': 'Residential',
    'R8P': 'Residential',
    'RA10P': 'Residential',
    'R-2.5': 'Residential',
    'R-10': 'Residential',
    'R4': 'Residential',
    'RS7.2': 'Residential',
    'R1': 'Residential',
    'R2': 'Residential',
    'R16': 'Residential',
    'R 5': 'Residential',
    'R 15': 'Residential',
    'R-1': 'Residential',
    'R-2': 'Residential',
    'RS 7200': 'Residential',
    'R1-7.2': 'Residential',
    'R-24': 'Residential',
    'R 11': 'Residential',
    'R1-7.5': 'Residential',
    'R 1': 'Residential',
    'RS 11.2': 'Residential',
    'R1-15': 'Residential',
    'RS 12.5': 'Residential',
    'R12': 'Residential',
    'R20': 'Residential',
    'R30': 'Residential',
    'R40': 'Residential',
    'R1 7.2': 'Residential',
    'R-3.5': 'Residential',
    'R1-5': 'Residential',
    'R-12': 'Residential',
    'R 8400': 'Residential',
    'R-1-10': 'Residential',
    'R 7200': 'Residential',
    'R-8': 'Residential',
    'R-4': 'Residential',
    'R-3': 'Residential',
    'R-1.8': 'Residential',
    'R 12': 'Residential',
    'R 5400d': 'Residential',
    'R 5400a': 'Residential',
    'R 2800': 'Residential',
    'R 9600, SSHO': 'Residential',
    'R 2800, OP, CB': 'Residential',
    'R 4000, OP, CB': 'Residential',
    'R 4000, OP, NB': 'Residential',
    'R 2800, OP, LI': 'Residential',
    'R 5400d, SSHO': 'Residential',
    'R 4000': 'Residential',
    'R 8400, OP': 'Residential',
    'R 9600, OP': 'Residential',
    'R 4000, OP, LI': 'Residential',
    'R-18': 'Residential',
    'R-20': 'Residential',
    'R-7.2': 'Residential',
    'R-10/C': 'Residential',

    # Manufacturing categories
    'M1': 'Manufacturing',
    'M2': 'Manufacturing',
    'MFR': 'Manufacturing',
    'MU': 'Manufacturing',
    'MUR': 'Manufacturing',
    'MIO': 'Manufacturing',
    'MUR-70': 'Manufacturing',
    'MUR-45': 'Manufacturing',
    'MUR-35': 'Manufacturing',
    'MR': 'Manufacturing',
    'MR-E': 'Manufacturing',
    'MR-M': 'Manufacturing',
    'MR-RC': 'Manufacturing',
    'MR-G': 'Manufacturing',
    'MR (M)': 'Manufacturing',
    'MR (M1)': 'Manufacturing',
    'MR (M2)': 'Manufacturing',
    'RC': 'Manufacturing',
    'RM': 'Manufacturing',
    'RM2.4': 'Manufacturing',
    'RM1.8': 'Manufacturing',
    'RM-F': 'Manufacturing',
    'RMH': 'Manufacturing',
    'RM900': 'Manufacturing',
    'RM900P': 'Manufacturing',
    'RM2400': 'Manufacturing',
    'RM3600': 'Manufacturing',
    'RM 3.6': 'Manufacturing',
    'RM 50': 'Manufacturing',
    'RM-1800': 'Manufacturing',
    'RML': 'Manufacturing',
    'RM 5.0': 'Manufacturing',
    'RML': 'Manufacturing',
    'RM 900': 'Manufacturing',
    'MIO-50-NC2-40': 'Manufacturing',
    'MIO-37-LR2 (M)': 'Manufacturing',
    'MIO-37-LR1 (M)': 'Manufacturing',
    'UM-2400': 'Manufacturing',
    'UM-3600': 'Manufacturing',
    'UD': 'Manufacturing',
    'UDC': 'Manufacturing',
    'UDH': 'Manufacturing',
    'UO': 'Manufacturing',

    # Transport, Communications, and Public Services
    'UR': 'Transport, Communications, and Public Services',
    'URP': 'Transport, Communications, and Public Services',
    'UP': 'Transport, Communications, and Public Services',
    'UPZ': 'Transport, Communications, and Public Services',
    'U1': 'Transport, Communications, and Public Services',
    'U2': 'Transport, Communications, and Public Services',
    'U3': 'Transport, Communications, and Public Services',
    'U4': 'Transport, Communications, and Public Services',
    'U5': 'Transport, Communications, and Public Services',
    'U6': 'Transport, Communications, and Public Services',
    'U7': 'Transport, Communications, and Public Services',
    'U8': 'Transport, Communications, and Public Services',
    'U9': 'Transport, Communications, and Public Services',
    'U10': 'Transport, Communications, and Public Services',
    'U11': 'Transport, Communications, and Public Services',
    'U12': 'Transport, Communications, and Public Services',
    'U13': 'Transport, Communications, and Public Services',
    'U14': 'Transport, Communications, and Public Services',
    'U15': 'Transport, Communications, and Public Services',
    'U16': 'Transport, Communications, and Public Services',
    'U17': 'Transport, Communications, and Public Services',
    'U18': 'Transport, Communications, and Public Services',
    'U19': 'Transport, Communications, and Public Services',
    'U20': 'Transport, Communications, and Public Services',
    'U21': 'Transport, Communications, and Public Services',
    'U22': 'Transport, Communications, and Public Services',
    'U23': 'Transport, Communications, and Public Services',
    'U24': 'Transport, Communications, and Public Services',
    'U25': 'Transport, Communications, and Public Services',
    'U26': 'Transport, Communications, and Public Services',
    'U27': 'Transport, Communications, and Public Services',
    'U28': 'Transport, Communications, and Public Services',
    'U29': 'Transport, Communications, and Public Services',
    'U30': 'Transport, Communications, and Public Services',
    'U31': 'Transport, Communications, and Public Services',
    'U32': 'Transport, Communications, and Public Services',
    'U33': 'Transport, Communications, and Public Services',
    'U34': 'Transport, Communications, and Public Services',
    'U35': 'Transport, Communications, and Public Services',
    'U36': 'Transport, Communications, and Public Services',
    'U37': 'Transport, Communications, and Public Services',
    'U38': 'Transport, Communications, and Public Services',
    'U39': 'Transport, Communications, and Public Services',
    'U40': 'Transport, Communications, and Public Services',
    'U41': 'Transport, Communications, and Public Services',
    'U42': 'Transport, Communications, and Public Services',
    'U43': 'Transport, Communications, and Public Services',
    'U44': 'Transport, Communications, and Public Services',
    'U45': 'Transport, Communications, and Public Services',
    'U46': 'Transport, Communications, and Public Services',
    'U47': 'Transport, Communications, and Public Services',
    'U48': 'Transport, Communications, and Public Services',
    'U49': 'Transport, Communications, and Public Services',
    'U50': 'Transport, Communications, and Public Services',
    'U51': 'Transport, Communications, and Public Services',
    'U52': 'Transport, Communications, and Public Services',
    'U53': 'Transport, Communications, and Public Services',
    'U54': 'Transport, Communications, and Public Services',
    'U55': 'Transport, Communications, and Public Services',
    'U56': 'Transport, Communications, and Public Services',
    'U57': 'Transport, Communications, and Public Services',
    'U58': 'Transport, Communications, and Public Services',
    'U59': 'Transport, Communications, and Public Services',
    'U60': 'Transport, Communications, and Public Services',
    'U61': 'Transport, Communications, and Public Services',
    'U62': 'Transport, Communications, and Public Services',
    'U63': 'Transport, Communications, and Public Services',
    'U64': 'Transport, Communications, and Public Services',
    'U65': 'Transport, Communications, and Public Services',
    'U66': 'Transport, Communications, and Public Services',
    'U67': 'Transport, Communications, and Public Services',
    'U68': 'Transport, Communications, and Public Services',
    'U69': 'Transport, Communications, and Public Services',
    'U70': 'Transport, Communications, and Public Services',
    'U71': 'Transport, Communications, and Public Services',
    'U72': 'Transport, Communications, and Public Services',
    'U73': 'Transport, Communications, and Public Services',
    'U74': 'Transport, Communications, and Public Services',
    'U75': 'Transport, Communications, and Public Services',
    'U76': 'Transport, Communications, and Public Services',
    'U77': 'Transport, Communications, and Public Services',
    'U78': 'Transport, Communications, and Public Services',
    'U79': 'Transport, Communications, and Public Services',
    'U80': 'Transport, Communications, and Public Services',
    'U81': 'Transport, Communications, and Public Services',
    'U82': 'Transport, Communications, and Public Services',
    'U83': 'Transport, Communications, and Public Services',
    'U84': 'Transport, Communications, and Public Services',
    'U85': 'Transport, Communications, and Public Services',
    'U86': 'Transport, Communications, and Public Services',
    'U87': 'Transport, Communications, and Public Services',
    'U88': 'Transport, Communications, and Public Services',
    'U89': 'Transport, Communications, and Public Services',
    'U90': 'Transport, Communications, and Public Services',
    'U91': 'Transport, Communications, and Public Services',
    'U92': 'Transport, Communications, and Public Services',
    'U93': 'Transport, Communications, and Public Services',
    'U94': 'Transport, Communications, and Public Services',
    'U95': 'Transport, Communications, and Public Services',
    'U96': 'Transport, Communications, and Public Services',
    'U97': 'Transport, Communications, and Public Services',
    'U98': 'Transport, Communications, and Public Services',
    'U99': 'Transport, Communications, and Public Services',

    # Recreational uses
    'R-P': 'Recreational',
    'R-M': 'Recreational',
    'R-N': 'Recreational',
    'R-G': 'Recreational',
    'R-E': 'Recreational',
    'R-O': 'Recreational',

    # Parks
    'P': 'Parks',
    'PR': 'Parks',
    'PR1': 'Parks',
    'PR2': 'Parks',
    'PR3': 'Parks',

    # Commercial
    'C': 'Commercial',
    'CR': 'Commercial',
    'C1': 'Commercial',
    'C2': 'Commercial',
    'C3': 'Commercial',
    'CM': 'Commercial',

    # Other
    'OS': 'Other',
    'PUD': 'Other',
    'G': 'Other',
    'G-R': 'Other',
    'O': 'Other',
    'O1': 'Other',
    'O2': 'Other',
    'O3': 'Other',
    'O4': 'Other',
    'O5': 'Other',
    'O6': 'Other',
    'O7': 'Other',
    'O8': 'Other',
    'O9': 'Other',
    'OS1': 'Other',
    'OS2': 'Other',
    'OS3': 'Other',
    'OS4': 'Other',
    'OS5': 'Other',
    'OS6': 'Other',
    'OS7': 'Other',
    'OS8': 'Other',
    'OS9': 'Other',
    'OS10': 'Other',
    'OS11': 'Other',
    'OS12': 'Other',
    'OS13': 'Other',
    'OS14': 'Other',
    'OS15': 'Other',
    'OS16': 'Other',
    'OS17': 'Other',
    'OS18': 'Other',
    'OS19': 'Other',
    'OS20': 'Other',
    'OS21': 'Other',
    'OS22': 'Other',
    'OS23': 'Other',
    'OS24': 'Other',
    'OS25': 'Other',
    'OS26': 'Other',
    'OS27': 'Other',
    'OS28': 'Other',
    'OS29': 'Other',
    'OS30': 'Other',
    'OS31': 'Other',
    'OS32': 'Other',
    'OS33': 'Other',
    'OS34': 'Other',
    'OS35': 'Other',
    'OS36': 'Other',
    'OS37': 'Other',
    'OS38': 'Other',
    'OS39': 'Other',
    'OS40': 'Other',
    'OS41': 'Other',
    'OS42': 'Other',
    'OS43': 'Other',
    'OS44': 'Other',
    'OS45': 'Other',
    'OS46': 'Other',
    'OS47': 'Other',
    'OS48': 'Other',
    'OS49': 'Other',
    'OS50': 'Other',
    'OS51': 'Other',
    'OS52': 'Other',
    'OS53': 'Other',
    'OS54': 'Other',
    'OS55': 'Other',
    'OS56': 'Other',
    'OS57': 'Other',
    'OS58': 'Other',
    'OS59': 'Other',
    'OS60': 'Other',
}

In [24]:
kingco_sales_df['zoning_type'] = kingco_sales_df['zoning'].map(zoning_to_type).fillna('Unknown')
kingco_homes_df['zoning_type'] = kingco_homes_df['zoning'].map(zoning_to_type).fillna('Unknown')

In [25]:
kingco_sales_df = kingco_sales_df.drop(columns=['zoning'])
kingco_homes_df = kingco_homes_df.drop(columns=['zoning'])

# Load

In [26]:
table_name = "kingco_sales"
table_name2 = "kingco_homes"
table_name3 = "kingco_index"

In [27]:
kingco_sales_df.to_sql(table_name, engine, if_exists='replace', index=False)
kingco_homes_df.to_sql(table_name2, engine, if_exists='replace', index=False)
kingco_index.to_sql(table_name3, engine, if_exists='replace', index=False)

49